# Imports

In [ ]:
!pip install codecarbon
!pip install keras-flops

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10, mnist
from keras.preprocessing.image import img_to_array, array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Model
from codecarbon import EmissionsTracker
from keras_flops import get_flops

# Input Data and Data Augmentation

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions and classes
input_shape = x_train.shape[1:]
num_classes = 10

# Target one-hot vectorization
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(input_shape)

In [ ]:
# Load the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.dstack([x_train] * 3)
x_train = x_train.reshape(-1, 28,28,3)
x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_train])
x_test = np.dstack([x_test] * 3)
x_test = x_test.reshape(-1, 28,28,3)
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in x_test])

# Input image dimensions and classes
input_shape = x_train.shape[1:]
num_classes = 10

# Target one-hot vectorization
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(input_shape)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
)  # randomly flip images
        
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Models

In [ ]:
# Parameters 
batch_size = 32
epochs = 50
optimizer = 'adam'

## VGG16

In [ ]:
# Import pre-trained layers
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_vgg16.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_vgg16.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# This is the model we will train
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=predictions)
model_vgg16.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg16.summary()

## VGG19


In [ ]:
# Import pre-trained layers
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_vgg19.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_vgg19.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=predictions)
model_vgg19.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg19.summary()

## ResNet

In [ ]:
# Import pre-trained layers
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape) 
 
# Freeze convolutional layers
for layer in base_model_resnet50.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model_resnet50.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(4096, activation='relu')(x) # number of layers and units are hyperparameters, as usual
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x) # should match # of classes predicted

# This is the model we will train
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=predictions)
model_resnet50.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.summary()

# [kera-flops](https://pypi.org/project/keras-flops/)

In [ ]:
# Models
flops = get_flops(model_vgg16, batch_size=batch_size)
print(f"FLOPS VGG16:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_vgg19, batch_size=batch_size)
print(f"FLOPS VGG19:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_resnet50, batch_size=batch_size)
print(f"FLOPS ResNet50: {flops / 10 ** 9:.03} G - {flops}")

In [ ]:
# Trainable Layer
flops = get_flops(model_vgg16, batch_size=batch_size) - get_flops(base_model_vgg16, batch_size=batch_size)
print(f"FLOPS VGG16:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_vgg19, batch_size=batch_size) - get_flops(base_model_vgg19, batch_size=batch_size)
print(f"FLOPS VGG19:    {flops / 10 ** 9:.03} G - {flops}")

flops = get_flops(model_resnet50, batch_size=batch_size) - get_flops(base_model_resnet50, batch_size=batch_size)
print(f"FLOPS ResNet50: {flops / 10 ** 9:.03} G - {flops}")

# [Codecarbon](https://github.com/mlco2/codecarbon)

In [ ]:
tracker = EmissionsTracker()

# Train VGG16
for i in range(3):
  tracker.start()
  history_vgg16 = model_vgg16.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()

In [ ]:
tracker = EmissionsTracker()

# Train VGG19
for i in range(3):
  tracker.start()
  history_vgg19 = model_vgg19.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()

In [ ]:
tracker = EmissionsTracker()

# Train ResNet50
for i in range(3):
  tracker.start()
  history_resnet50 = model_resnet50.fit_generator(
      datagen.flow(x_train, y_train, batch_size=batch_size), 
      epochs=epochs,
      verbose = 1,
      validation_data = (x_test, y_test)
  )
  tracker.stop()